In [1]:
import copernicusmarine as cm
import xarray as xr
import polars as pl
import numpy as np
from datetime import datetime
import sys
import os
sys.path.append(os.path.abspath("../python/utils"))
from get_data import make_request_and_open_copernicus

/home/pablo/miniconda3/envs/proy3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
temp = pl.read_csv("../../data/temporal_subset.csv")
box = pl.read_csv("../../data/world_box.csv")

In [3]:
day = temp.filter(pl.col("mas_actual") == 0)["day"].to_list()[0]
month = temp.filter(pl.col("mas_actual") == 0)["month"].to_list()[0]
year = temp.filter(pl.col("mas_actual") == 0)["year"].to_list()[0]
min_time = f"{day}-{month}-{year}"
day = temp.filter(pl.col("mas_actual") == 1)["day"].to_list()[0]
month = temp.filter(pl.col("mas_actual") == 1)["month"].to_list()[0]
year = temp.filter(pl.col("mas_actual") == 1)["year"].to_list()[0]
max_time = f"{day}-{month}-{year}"
fecha_min = datetime.strptime(min_time, "%d-%m-%Y")
fecha_min_format = fecha_min.strftime("%Y-%m-%dT%H:%M:%S")
fecha_max = datetime.strptime(max_time, "%d-%m-%Y")
fecha_max_format = fecha_max.strftime("%Y-%m-%dT%H:%M:%S")
print("min", fecha_min_format)
print("max", fecha_max_format)

min 1999-06-15T00:00:00
max 2015-10-07T00:00:00


## Fecha mínima provisional

Para trabajar con un archivo más manejable, usaremos un subset más pequeño y luego al final usaremos el completo.

In [4]:
fecha_min_provisional = "9-10-2015"
fecha_min_provisional = datetime.strptime(fecha_min_provisional, "%d-%m-%Y")
fecha_min_format = fecha_min_provisional.strftime("%Y-%m-%dT%H:%M:%S")

In [5]:
max_latitude = box["max_latitude"].to_list()[0]
min_latitude = box["min_latitude"].to_list()[0]
max_longitude = box["max_longitude"].to_list()[0]
min_longitude = box["min_longitude"].to_list()[0]
print("max", max_latitude, max_longitude)
print("min", min_latitude, min_longitude)

max -61.767468 179.988159
min -78.173 -179.998241


## Request

In [ ]:
ds = make_request_and_open_copernicus("cmems_mod_glo_phy_my_0.083deg_P1M-m", "Global_Ocean_Physics_Reanalysis.nc",
                                        fecha_min_format, fecha_max_format, max_latitude, min_latitude, max_longitude, min_longitude)
ds

INFO - 2025-03-14T10:57:58Z - Downloading Copernicus Marine data requires a Copernicus Marine username and password, sign up for free at: https://data.marine.copernicus.eu/register


Copernicus Marine username:Copernicus Marine password:

INFO - 2025-03-14T10:58:29Z - Selected dataset version: "202311"
INFO - 2025-03-14T10:58:29Z - Selected dataset part: "default"
WARNING - 2025-03-14T10:58:34Z - Some of your subset selection [0, 1] for the depth dimension exceed the dataset coordinates [0.49402499198913574, 5727.9169921875]
INFO - 2025-03-14T10:58:38Z - Starting download. Please wait...
100%|██████████| 66/66 [00:01<00:00, 34.70it/s]
INFO - 2025-03-14T10:58:41Z - Successfully downloaded to /home/pablo/Desktop/zird/2/proy/github/data/copernicus/Global_Ocean_Physics_Reanalysis_(2).nc


<xarray.Dataset> Size: 75MB
Dimensions:    (depth: 1, latitude: 197, longitude: 4319, time: 1)
Coordinates:
  * depth      (depth) float32 4B 0.494
  * latitude   (latitude) float32 788B -78.17 -78.08 -78.0 ... -61.92 -61.83
  * longitude  (longitude) float32 17kB -179.9 -179.8 -179.8 ... 179.8 179.9
  * time       (time) datetime64[ns] 8B 2015-10-01
Data variables:
    bottomT    (time, latitude, longitude) float64 7MB ...
    mlotst     (time, latitude, longitude) float64 7MB ...
    siconc     (time, latitude, longitude) float64 7MB ...
    sithick    (time, latitude, longitude) float64 7MB ...
    so         (time, depth, latitude, longitude) float64 7MB ...
    thetao     (time, depth, latitude, longitude) float64 7MB ...
    uo         (time, depth, latitude, longitude) float64 7MB ...
    usi        (time, latitude, longitude) float64 7MB ...
    vo         (time, depth, latitude, longitude) float64 7MB ...
    vsi        (time, latitude, longitude) float64 7MB ...
    zos        (time, latitude, longitude) float64 7MB ...
Attributes:
    references:                http://marine.copernicus.eu
    contact:                   servicedesk.cmems@mercator-ocean.eu
    credit:                    E.U. Copernicus Marine Service Information (CM...
    institution:               Mercator Ocean
    title:                     Monthly mean fields for product GLOBAL_REANALY...
    producer:                  CMEMS - Global Monitoring and Forecasting Centre
    source:                    MERCATOR GLORYS12V1
    Conventions:               CF-1.6
    copernicusmarine_version:  2.0.1

In [ ]:
lat_bins = np.arange(ds.latitude.min(), ds.latitude.max() + 1, 1)
lon_bins = np.arange(ds.longitude.min(), ds.longitude.max() + 1, 1)

lat_labels = (lat_bins[:-1] + lat_bins[1:]) / 2
lon_labels = (lon_bins[:-1] + lon_bins[1:]) / 2

ds_binned = ds.groupby_bins("latitude", lat_bins, labels=lat_labels).mean()
ds_binned = ds_binned.groupby_bins("longitude", lon_bins, labels=lon_labels).mean()
print(ds_binned)

<xarray.Dataset> Size: 542kB
Dimensions:         (longitude_bins: 360, latitude_bins: 17, time: 1, depth: 1)
Coordinates:
  * depth           (depth) float32 4B 0.494
  * time            (time) datetime64[ns] 8B 2015-10-01
  * latitude_bins   (latitude_bins) object 136B (-78.167, -77.167] ... (-62.1...
  * longitude_bins  (longitude_bins) object 3kB (-179.917, -178.917] ... (179...
Data variables:
    bottomT         (longitude_bins, latitude_bins, time) float64 49kB nan .....
    mlotst          (longitude_bins, latitude_bins, time) float64 49kB nan .....
    siconc          (longitude_bins, latitude_bins, time) float64 49kB nan .....
    sithick         (longitude_bins, latitude_bins, time) float64 49kB nan .....
    so              (longitude_bins, latitude_bins, time, depth) float64 49kB ...
    thetao          (longitude_bins, latitude_bins, time, depth) float64 49kB ...
    uo              (longitude_bins, latitude_bins, time, depth) float64 49kB ...
    usi             (longitud

In [39]:
ds_binned

<xarray.Dataset> Size: 542kB
Dimensions:         (longitude_bins: 360, latitude_bins: 17, time: 1, depth: 1)
Coordinates:
  * depth           (depth) float32 4B 0.494
  * time            (time) datetime64[ns] 8B 2015-10-01
  * latitude_bins   (latitude_bins) object 136B (-78.167, -77.167] ... (-62.1...
  * longitude_bins  (longitude_bins) object 3kB (-179.917, -178.917] ... (179...
Data variables:
    bottomT         (longitude_bins, latitude_bins, time) float64 49kB nan .....
    mlotst          (longitude_bins, latitude_bins, time) float64 49kB nan .....
    siconc          (longitude_bins, latitude_bins, time) float64 49kB nan .....
    sithick         (longitude_bins, latitude_bins, time) float64 49kB nan .....
    so              (longitude_bins, latitude_bins, time, depth) float64 49kB ...
    thetao          (longitude_bins, latitude_bins, time, depth) float64 49kB ...
    uo              (longitude_bins, latitude_bins, time, depth) float64 49kB ...
    usi             (longitude_bins, latitude_bins, time) float64 49kB nan .....
    vo              (longitude_bins, latitude_bins, time, depth) float64 49kB ...
    vsi             (longitude_bins, latitude_bins, time) float64 49kB nan .....
    zos             (longitude_bins, latitude_bins, time) float64 49kB nan .....
Attributes:
    references:                http://marine.copernicus.eu
    contact:                   servicedesk.cmems@mercator-ocean.eu
    credit:                    E.U. Copernicus Marine Service Information (CM...
    institution:               Mercator Ocean
    title:                     Monthly mean fields for product GLOBAL_REANALY...
    producer:                  CMEMS - Global Monitoring and Forecasting Centre
    source:                    MERCATOR GLORYS12V1
    Conventions:               CF-1.6
    copernicusmarine_version:  2.0.1